In [ ]:
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

def get_top_30_stocks(rebal_month):
    if rebal_month.lower() == 'june':
        rebal_date = '2024-05-31'
    elif rebal_month.lower() == 'december':
        rebal_date = '2024-11-30'
    else:
        raise ValueError("Invalid rebalancing month. Please enter 'June' or 'December'.")

    df = pd.read_csv('ind_nifty200list.csv')
    stock_tickers = df['Symbol'].tolist()
    stock_tickers = [ticker + '.NS' for ticker in stock_tickers]
    stock_names = df['Company Name'].tolist()
    stock_data = {}

    for ticker, name in zip(stock_tickers, stock_names):
        stock = yf.Ticker(ticker)
        hist = stock.history(start='2022-06-01', end='2024-05-31', interval='1d')
        stock_data[name] = hist['Close']
    stock_data = pd.DataFrame(stock_data)
    stock_data.index = pd.to_datetime(stock_data.index).date

    logstd = {}
    mr12 = {}
    mr6 = {}
    for name in stock_names:

        rebal_date_dt = pd.to_datetime(rebal_date).date()
        start_date = (pd.to_datetime(rebal_date) - pd.DateOffset(years=1)).date()
        log_returns = np.log(stock_data[name].loc[start_date:rebal_date_dt] / stock_data[name].loc[start_date:rebal_date_dt].shift(1))
        logstd[name] = log_returns.std()
        pricem1 = stock_data[name].loc[stock_data.index <= rebal_date_dt].iloc[-1]
        pricem13 = stock_data[name].loc[stock_data.index <= (pd.to_datetime(rebal_date) - pd.DateOffset(months=12)).date()].iloc[-1]
        pricem7 = stock_data[name].loc[stock_data.index <= (pd.to_datetime(rebal_date) - pd.DateOffset(months=6)).date()].iloc[-1]
        mr12[name] = ((pricem1 / pricem13) - 1) / logstd[name]
        mr6[name] = ((pricem1 / pricem7) - 1) / logstd[name]

    meanmr12 = pd.Series(mr12).mean()
    meanmr6 = pd.Series(mr6).mean()

    z12 = {}
    z6 = {}
    weightedZ = {}
    normalizedZ = {}
    for name in stock_names:
        z12[name] = (mr12[name] - meanmr12) / logstd[name]
        z6[name] = (mr6[name] - meanmr6) / logstd[name]
        weightedZ[name] = z12[name] * 0.5 + z6[name] * 0.5
        normalizedZ[name] = 1 + weightedZ[name] if weightedZ[name] > 0 else (1 - weightedZ[name])**(-1)

    normalizedZ = pd.Series(normalizedZ)
    top30 = normalizedZ.sort_values(ascending=False)
    top30 = top30[0:30]
    top30 = top30.sort_index()

    return top30

def display_top_30_stocks():
    rebal_month_input = widgets.Text(
        value='June',
        placeholder='Enter rebalancing month (June or December)',
        description='Rebal Month:',
        disabled=False
    )

    display_button = widgets.Button(
        description='Get Top 30 Stocks',
        disabled=False,
        button_style='',
        tooltip='Click to get top 30 stocks',
        icon='check'
    )

    output = widgets.Output()

    def on_button_click(b):
        with output:
            output.clear_output()
            rebal_month = rebal_month_input.value
            try:
                top_30_stocks = get_top_30_stocks(rebal_month)
                print(top_30_stocks)
            except ValueError as e:
                print(e)

    display_button.on_click(on_button_click)

    display(rebal_month_input, display_button, output)

display_top_30_stocks()


Text(value='June', description='Rebal Month:', placeholder='Enter rebalancing month (June or December)')

Button(description='Get Top 30 Stocks', icon='check', style=ButtonStyle(), tooltip='Click to get top 30 stocks…

Output()